In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Repo

In [2]:
!pip uninstall -y ultralytics

In [3]:
!git clone https://github.com/Umar-Farooq-2112/ultralytics-custom-backbone-neck.git
#!pip install --no-deps -e .

Cloning into 'ultralytics-custom-backbone-neck'...
remote: Enumerating objects: 45609, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 45609 (delta 42), reused 17 (delta 17), pack-reused 45534 (from 2)
Receiving objects: 100% (45609/45609), 27.35 MiB | 28.54 MiB/s, done.
Resolving deltas: 100% (33569/33569), done.


In [4]:
%cd ultralytics-custom-backbone-neck

/kaggle/working/ultralytics-custom-backbone-neck


# Dataset Preprocessing

In [5]:
os.environ['KAGGLE_USERNAME'] = "umarfarooq211203"
os.environ['KAGGLE_KEY'] = "d09191f7a1916f38b5f1fc8a385b5750"


In [6]:
!kaggle datasets download umarfarooq211203/defects-in-timber

Dataset URL: https://www.kaggle.com/datasets/umarfarooq211203/defects-in-timber
License(s): CC0-1.0
 69%|███████████████████████████▋            | 303M/438M [00:00<00:00, 1.58GB/s]
100%|████████████████████████████████████████| 438M/438M [00:00<00:00, 1.58GB/s]


In [7]:
import zipfile
import os

# Path to your zip file
zip_path = '/kaggle/working/ultralytics-custom-backbone-neck/defects-in-timber.zip'

# Directory where you want to extract the files
extract_dir = '/kaggle/working/ultralytics-custom-backbone-neck/defects-in-timber'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Open the zip file and extract all contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")


Files extracted to /kaggle/working/ultralytics-custom-backbone-neck/defects-in-timber


# Data Evaluation

In [8]:
import os

DATASET_PATH = "/kaggle/working/ultralytics-custom-backbone-neck/defects-in-timber"
splits = ["train", "valid", "test"]

NUM_CLASSES = 4

def count_images_per_class(split):
    label_dir = os.path.join(DATASET_PATH, split, "labels")

    img_count = {i: 0 for i in range(NUM_CLASSES)}

    for lbl in os.listdir(label_dir):
        if not lbl.endswith(".txt"):
            continue
        
        classes_in_file = set()
        with open(os.path.join(label_dir, lbl)) as f:
            for line in f:
                parts = line.strip().split()
                try:
                    cls = int(float(parts[0]))    # SAFE PARSE
                    classes_in_file.add(cls)
                except:
                    continue

        for cls in classes_in_file:
            img_count[cls] += 1

    return img_count


def count_objects_per_class(split):
    label_dir = os.path.join(DATASET_PATH, split, "labels")

    obj_count = {i: 0 for i in range(NUM_CLASSES)}

    for lbl in os.listdir(label_dir):
        if not lbl.endswith(".txt"):
            continue
        with open(os.path.join(label_dir, lbl)) as f:
            for line in f:
                parts = line.strip().split()
                try:
                    cls = int(float(parts[0]))
                    obj_count[cls] += 1
                except:
                    continue

    return obj_count


# ---- PRINT COUNTS ----
for split in splits:
    print(f"\n==== {split.upper()} ====")
    print("Images per class:", count_images_per_class(split))
    print("Objects per class:", count_objects_per_class(split))



==== TRAIN ====
Images per class: {0: 2628, 1: 2772, 2: 3431, 3: 2866}
Objects per class: {0: 3246, 1: 3897, 2: 5431, 3: 3445}

==== VALID ====
Images per class: {0: 365, 1: 376, 2: 488, 3: 406}
Objects per class: {0: 454, 1: 510, 2: 778, 3: 494}

==== TEST ====
Images per class: {0: 254, 1: 263, 2: 305, 3: 269}
Objects per class: {0: 307, 1: 357, 2: 506, 3: 332}


## Tiny Vit Backbone Architecture

## Build Model

In [9]:
!ls

check_model_size.py		   LICENSE
CITATION.cff			   mkdocs.yml
COMMON_ERRORS_AND_FIXES.md	   MOBILENETV3_YOLO_README.md
comprehensive_test.py		   MODEL_ENHANCEMENT_SUMMARY.md
CONTRIBUTING.md			   PRODUCTION_READY.md
CUSTOM_MODEL_DEVELOPMENT_GUIDE.md  pyproject.toml
CUSTOM_MODEL_README.md		   QUICKSTART_MOBILENETV3_YOLO.md
debug_output.py			   quick_test.py
defects-in-timber		   README.md
defects-in-timber.zip		   README.zh-CN.md
docker				   test_integration.py
docs				   test.py
examples			   tests
FILE_UPDATE_REFERENCE.md	   test_training.py
final_validation.py		   train_custom_model.py
integration_diagram.py		   TRAINING_COMPLETE_GUIDE.md
INTEGRATION_SUMMARY.md		   train_mobilenetv3_yolo.py
learn_custom_model.md		   ultralytics


In [10]:
from ultralytics import YOLO

# Load custom model
model = YOLO('ultralytics/cfg/models/custom/mobilenetv3-yolo.yaml')


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 108MB/s]


In [11]:
def print_model_params(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params

    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Non-trainable parameters: {non_trainable_params:,}")

# Example usage
print_model_params(model.model)  # model.model is the YOLOv8 backbone+neck+head


Total parameters: 4,239,152
Trainable parameters: 4,239,136
Non-trainable parameters: 16


In [12]:
from torchinfo import summary

# Assume input image size 640x640 with 3 channels
summary(model.model)
#summary(model.model, input_size=(1, 3, 640, 640))


Layer (type:depth-idx)                                                      Param #
MobileNetV3YOLO                                                             --
├─MobileNetV3BackboneDW: 1-1                                                --
│    └─Sequential: 2-1                                                      --
│    │    └─Conv2dNormActivation: 3-1                                       464
│    │    └─InvertedResidual: 3-2                                           744
│    │    └─InvertedResidual: 3-3                                           3,864
│    └─Sequential: 2-2                                                      --
│    │    └─InvertedResidual: 3-4                                           5,416
│    │    └─InvertedResidual: 3-5                                           13,736
│    │    └─InvertedResidual: 3-6                                           57,264
│    │    └─InvertedResidual: 3-7                                           57,264
│    └─Sequential: 2-3     

## Training

In [13]:
model.train(
    data="defects-in-timber/data.yaml",
    imgsz=640,
    batch=16,
    epochs=500,
    optimizer="SGD",          # switch to SGD
    momentum=0.937,           # typical momentum value for YOLO training
    weight_decay=5e-4,        # regularization
    lr0=0.01,                 # initial learning rate
    lrf=0.01,                 # final learning rate factor for cosine decay
    warmup_epochs=5,          # warmup for first 5 epochs
    patience=25,              # early stopping patience
    workers=4,
    device=0,
)


Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=defects-in-timber/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/models/custom/mobilenetv3-yolo.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=25,

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model summary: 487 layers, 4,209,740 parameters, 4,209,724 gradients
Model summary: 487 layers, 4,209,740 parameters, 4,209,724 gradients
Freezing layer 'head.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1576.7±550.2 MB/s, size: 64.5 KB)
train: Scanning /kaggle/working/ultralytics-custom-backbone-neck/defects-in-timber/train/labels... 6688 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 6688/6688 1.4Kit/s 4.6s<0.0s
train: New cache created: /kaggle/working/ultralytics-custom-backbone-neck/defects-in-timber/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 6, len(boxes) = 16019. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGra

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Memory Efficient attention defaults to a non-deterministic algorithm. To explicitly enable determinism call torch.use_deterministic_algorithms(True, warn_only=False). (Triggered internally at /pytorch/aten/src/ATen/native/transformers/cuda/attention_backward.cu:683.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/500      4.29G      4.058      4.683      3.623         58        640: 100% ━━━━━━━━━━━━ 418/418 3.2it/s 2:11<0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 30/30 3.3it/s 9.0s0.3s
                   all        930       2236      0.379      0.169     0.0786     0.0238

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/500      4.85G      3.421       3.28      3.206         53        640: 0% ──────────── 0/418  0.3s

...

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    114/500      6.65G      1.409     0.9751      1.362         61        640: 100% ━━━━━━━━━━━━ 418/418 3.6it/s 1:55<0.3ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 30/30 4.0it/s 7.4s0.3s
                   all        930       2236      0.758      0.779      0.788      0.458

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    115/500      6.67G      1.453      1.012      1.421         54        640: 0% ──────────── 0/418  0.3s

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    115/500      6.67G      1.404     0.9851      1.369         68        640: 24% ━━╸───────── 101/418 8.6it/s 27.8s<36.7s


KeyboardInterrupt: 

In [16]:
model = YOLO("/kaggle/working/ultralytics-custom-backbone-neck/runs/detect/train/weights/best.pt")

In [17]:
metrics = model.val(data="defects-in-timber/data.yaml", batch=16, imgsz=640)
print(metrics)

Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary: 487 layers, 4,209,740 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1295.7±316.7 MB/s, size: 40.5 KB)
val: Scanning /kaggle/working/ultralytics-custom-backbone-neck/defects-in-timber/valid/labels.cache... 930 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 930/930 2.0Mit/s 0.0s0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 3, len(boxes) = 2236. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 59/59 4.2it/s 14.0s0.2s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        930       2236      0.756      0.789      0.794      0.463
                 Crack        365        454       0.68      0.756      0.725      0.368
             Dead_Knot        376        510       0.69      0.735      0.714      0.388
             Live_Knot        488        778      0.794      0.775      0.808      0.421
       knot_with_crack        406        494      0.861      0.891      0.927      0.674
Speed: 0.9ms preprocess, 9.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/ultralytics-custom-backbone-neck/runs/detect/val3
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7917e19d7610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.0

In [18]:
# Evaluate on test dataset
test_metrics = model.val(data="defects-in-timber/data.yaml", split="test", batch=2, imgsz=640)
print(test_metrics)


Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary: 487 layers, 4,209,740 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1113.4±416.8 MB/s, size: 49.6 KB)
val: Scanning /kaggle/working/ultralytics-custom-backbone-neck/defects-in-timber/test/labels.cache... 627 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 627/627 1.5Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 314/314 22.4it/s 14.0s<0.0s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        627       1502      0.774      0.777        0.8      0.463
                 Crack        254        307      0.709      0.707      0.711      0.337
             Dead_Knot        263        357      0.738      0.767      0.758      0.426
             Live_Knot        305        506        0.8      0.737      0.806      0.416
       knot_with_crack        269        332      0.849      0.898      0.923      0.672
Speed: 1.0ms preprocess, 16.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /kaggle/working/ultralytics-custom-backbone-neck/runs/detect/val4
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7917e15b74d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.

In [ ]:
best_model = YOLO("./runs/detect/train/weights/best.pt")

In [ ]:
metrics = best_model.val(data="defects-in-timber/data.yaml", batch=16, imgsz=640)
print(metrics)

In [ ]:
# Evaluate on test dataset
test_metrics = best_model.val(data="defects-in-timber/data.yaml", split="test", batch=2, imgsz=640)
print(test_metrics)


In [19]:
import shutil
folder_path = "/kaggle/working/ultralytics-custom-backbone-neck/runs"  
archive_name = "/kaggle/working/yolov8s_transformer_results_full_train"  
if os.path.exists(archive_name+".zip"):
    os.remove(archive_name+".zip")
shutil.make_archive(archive_name, 'zip', folder_path)


'/kaggle/working/yolov8s_transformer_results_full_train.zip'

In [ ]:
# Total parameters
total_params = sum(p.numel() for p in best_model.model.parameters())
print(f"Total parameters: {total_params}")

# Trainable parameters
trainable_params = sum(p.numel() for p in best_model.model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")


In [ ]:
from torchinfo import summary

# Assume input image size 640x640 with 3 channels
summary(model.model, input_size=(1, 3, 640, 640))


# Storing Results

In [ ]:
if os.path.exists(dataset_dir):
    shutil.rmtree(dataset_dir)
    print(f"Folder '{dataset_dir}' deleted successfully!")
else:
    print(f"Folder '{dataset_dir}' does not exist.")


In [ ]:
os.listdir(dataset_dir)

In [ ]:
        # Example of creating a metadata file
dataset_dir = "/kaggle/dataset/custom-yolo-sgd-try2-results"
os.makedirs(dataset_dir, exist_ok=True)
import json
metadata = {
    "title": "Custom YOLOv8n SGD-try2 with Tranformers",
    "id": f"{os.environ['KAGGLE_USERNAME']}/custom-yolo-sgd-try2-results",
    "licenses": [{"name": "CC0-1.0"}]  # <-- This is required
}
with open(os.path.join(dataset_dir, "dataset-metadata.json"), "w") as f:
    json.dump(metadata, f, indent=4)

In [ ]:
src_folder = "/kaggle/working/ultralytics-custom-backbone-neck/runs"
dst_folder = "/kaggle/dataset/custom-yolo-sgd-try2-results"

# Copy all files and subfolders
for item in os.listdir(src_folder):
    s = os.path.join(src_folder, item)
    d = os.path.join(dst_folder, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True)
    else:
        shutil.copy2(s, d)

print("Dataset files copied!")

In [ ]:
!kaggle datasets create -p "{dataset_dir}" --dir-mode zip

# Other Eval

In [ ]:
!kaggle datasets download umarfarooq211203/custom-gnn-yolov8s-results-full-train

In [ ]:
import zipfile
import os

# Path to your zip file
zip_path = '/kaggle/working/custom-gnn-yolov8s-results-full-train.zip'

# Directory where you want to extract the files
extract_dir = '/kaggle/working/runs'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Open the zip file and extract all contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")


In [ ]:
model = YOLO("/kaggle/working/runs/train/weights/best.pt")

In [ ]:
import torch
import time

model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

dummy_input = torch.rand(1, 3, 640, 640).to(device)

# Warm-up
for _ in range(10):
    with torch.no_grad():
        _ = model(dummy_input)

# Measure
iterations = 100
start = time.time()
for _ in range(iterations):
    with torch.no_grad():
        _ = model(dummy_input)
end = time.time()

avg_time = (end - start) / iterations
fps = 1 / avg_time

print(f"Average inference time: {avg_time*1000:.2f} ms")
print(f"FPS: {fps:.2f}")


In [ ]:
!pip install --no-deps thop

In [ ]:
# model is your trained YOLO model
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


In [ ]:
import os

model_path = "/kaggle/working/runs/detect/train/weights/best.pt"
size_in_MB = os.path.getsize(model_path) / 1e6
print(f"Model size: {size_in_MB:.2f} MB")
